# Deployment Pipeline
This pipeline is triggered when a new approved model is detected in the model registry.

## Setup

In [69]:
import boto3
import pandas as pd
import numpy as np
import time
import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup

In [70]:
%env AWS_PROFILE=aeroxye-sagemaker

env: AWS_PROFILE=aeroxye-sagemaker


In [71]:
!aws sts get-caller-identity

{
    "UserId": "AROAWC4YSIQL5OBFCNGEX:botocore-session-1687435302",
    "Account": "418542404631",
    "Arn": "arn:aws:sts::418542404631:assumed-role/SageMaker-UserRole/botocore-session-1687435302"
}


In [72]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SageMaker-UserRole')['Role']['Arn']

region = boto3.Session().region_name
print(f'Current region: {region}')

boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
sagemaker_client.list_feature_groups()

featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Current region: ap-southeast-1


In [73]:
from sagemaker.workflow.pipeline_context import PipelineSession
pipeline_session = PipelineSession()

model_type = "bpr"
pipeline_name = "petfinder6000-deploy-pipeline"  # SageMaker Pipeline name
model_package_group_name = f"PetFinder6000-{model_type}"

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


## Define pipeline parameters

In [74]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterString

default_bucket = sagemaker_session.default_bucket()
batch_name = 'batch-bpr-'
current_time = time.strftime("%Y%m%d-%H-%M-%S", time.gmtime())
run_name = f"{batch_name}{current_time}"

# prefixes
job_name = f"{pipeline_name}/batch-inference/{run_name}"

rank_file_name = f'ranking-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
ranking_path = f's3://petfinder6000/ranking/{rank_file_name}'
batch_input_path = f"{ranking_path}/batch-input"

lambda_function_name = "lambdaBatchTransformPipelineLambda"

# processing step parameters
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value="ml.m5.xlarge")
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

# inference step parameters
inference_instance_type = ParameterString(name="InferenceInstanceType", default_value="ml.m5.xlarge")
inference_image = ParameterString(name="InferenceImageURI", default_value='418542404631.dkr.ecr.ap-southeast-1.amazonaws.com/petfinder6000:cornac-39-inference-v15')
inference_instance_count = ParameterInteger(name="InferenceInstanceCount", default_value=1)

## Run Batch Inference

### Pull user and interactions data from feature store

In [75]:
from sagemaker.processing import FrameworkProcessor, ProcessingOutput
from sagemaker.sklearn import SKLearn

batch_input_processor = FrameworkProcessor(
    role=role,
    estimator_cls=SKLearn,
    framework_version='0.23-1',
    command=["python3"],
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
    sagemaker_session=pipeline_session,
    env={"REGION": region, "INFERENCE_PATH": batch_input_path},
)

batch_input_args = batch_input_processor.run(
    outputs=[
        ProcessingOutput(output_name="batch-inference-input", source="/opt/ml/processing/batch_input"),
    ],
    code="batch_input.py",
    source_dir="./scripts",
    job_name=job_name,
)

C:\Users\yongr\anaconda3\envs\cornac39\lib\site-packages\sagemaker\workflow\pipeline_context.py:286: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [76]:
from sagemaker.workflow.steps import ProcessingStep

step_get_batch_inputs = ProcessingStep(
    name="GetBatchInput",
    step_args=batch_input_args,
)

### Lambda Step to Load Model

In [77]:
from sagemaker.lambda_helper import Lambda

# Lambda helper class can be used to create the Lambda function
func = Lambda(
    function_name=lambda_function_name,
    execution_role_arn=role,
    script="get_approved_package.py",
    handler="get_approved_package.handler",
    timeout=600,
    memory_size=128,
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [78]:
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

step_latest_model_fetch = LambdaStep(
    name="FetchLatestApprovedModel",
    lambda_func=func,
    inputs={
        "model_package_group_name": model_package_group_name,
    },
    outputs=[
        LambdaOutput(output_name="ModelUrl", output_type=LambdaOutputTypeEnum.String),
        LambdaOutput(output_name="ImageUri", output_type=LambdaOutputTypeEnum.String),
    ],
)

In [79]:
from sagemaker.model import Model

model = Model(
    image_uri=step_latest_model_fetch.properties.Outputs["ImageUri"],
    model_data=step_latest_model_fetch.properties.Outputs["ModelUrl"],
    sagemaker_session=pipeline_session,
    role=role,
)

In [80]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep

inputs = CreateModelInput(
    instance_type=inference_instance_type,
)
step_create_model = CreateModelStep(
    name="CreateModel",
    model=model,
    inputs=inputs,
)

## Transformer

In [81]:
from sagemaker.transformer import Transformer

transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_count=inference_instance_count,
    instance_type=inference_instance_type,
    assemble_with='Line',
    output_path=ranking_path
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [82]:
from sagemaker.workflow.steps import TransformStep
from sagemaker.inputs import TransformInput

step_transform = TransformStep(
    name="BatchTransform",
    transformer=transformer,
    inputs=TransformInput(
        data=step_get_batch_inputs.properties.ProcessingOutputConfig.Outputs["batch-inference-input"].S3Output.S3Uri,
        split_type='Line',
        content_type='text/csv'
    )
)

## Create pipeline

In [83]:
from sagemaker.workflow.pipeline import Pipeline

# Create a Sagemaker Pipeline.
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        inference_instance_type,
        inference_image,
        inference_instance_count
    ],
    steps=[step_get_batch_inputs, step_latest_model_fetch, step_create_model, step_transform],
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [84]:
pipeline.upsert(role_arn=role)

Using provided s3_resource


INFO:sagemaker.processing:Uploaded ./scripts to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-deploy-pipeline/code/11c5ecb4a5056c83b40e8573b5378f36/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-deploy-pipeline/code/7d6fec0b9c14c20392989f9c2c803826/runproc.sh
INFO:sagemaker.processing:Uploaded ./scripts to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-deploy-pipeline/code/11c5ecb4a5056c83b40e8573b5378f36/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-deploy-pipeline/code/7d6fec0b9c14c20392989f9c2c803826/runproc.sh


Using provided s3_resource


{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-1:418542404631:pipeline/petfinder6000-deploy-pipeline',
 'ResponseMetadata': {'RequestId': '6a267f9a-ec0a-4143-96aa-f162fa6226cf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6a267f9a-ec0a-4143-96aa-f162fa6226cf',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Thu, 22 Jun 2023 12:04:55 GMT'},
  'RetryAttempts': 0}}

In [85]:
execution = pipeline.start()

In [18]:
execution.wait()